### Beispiel für das Modellieren mit Klassen

Bot-Programmierung [Codingame: Fantastic Bits](https://www.codingame.com/ide/puzzle/fantastic-bits)

<img src='fantastic.png'>

In [3]:
import sys
import itertools as it
import math

def log(*x):
    print(*x, file=sys.stderr)

class Vector:
    def __init__(self, x=0, y=0):
        ''' constructor '''
        self.x = x
        self.y = y
        
    def add(self, other):
        ''' returns result of other vector added to this vector '''
        return Vector(self.x+other.x, self.y+other.y)

    def sub(self, other, inplace=False):
        ''' returns result of other vector subtracted from this vector '''
        return Vector(self.x-other.x, self.y-other.y)

    def mult(self, k, inplace=False):
        ''' returs result of multiplying this vector by scalar k '''
        if not inplace:
            return Vector(k*self.x,k*self.y)
        self.x *=k
        self.y *=k
        
    def __str__(self):
        ''' returns String mit ganzzahligen Koordinaten '''
        return f'({self.x:.0f}/{self.y:.0f})'

    def dist(self, other):
        ''' returns Entfernung zwischen diesem und dem anderen Vektor '''
        return math.dist((self.x,self.y), (other.x, other.y))

class Gerade:
    def __init__(self, p, u):
        ''' constructor: p - Stützvektor, u - Richtungsvektor '''
        self.p = p
        self.u = u

    def schnittpunkt(self, other, mode1='Gerade', mode2='Gerade'):
        '''
        returns: Schnittpunkt zwischen dieser Geraden und einer anderen.
        mode1/2 kann sein: 'Gerade', 'Strahl', 'Strecke'.
        Bei Strahl ist p Ausgangspunkt und u die Richtung des Strahls
        Als Strecke wird der Bereich p bis p+u interpretiert.
        returns None, falls kein eindeutiges Ergebnis existiert. 
        '''
        p1, p2 = self.p.x, self.p.y
        u1, u2 = self.u.x, self.u.y
        p3, p4 = other.p.x, other.p.y
        u3, u4 = other.u.x, other.u.y

        def det(a,b,c,d):
            return a*d-b*c
        # Cramersche Regel
        nenner = det(u1,-u3,u2,-u4)
        if nenner == 0: return None   # keine eindeutige Lösung

        s = det(p3-p1, -u3, p4-p2, -u4)/nenner
        t = det(u1, p3-p1, u2, p4-p2)/nenner

        if mode1 == 'Strahl' and s < 0: return None
        if mode2 == 'Strahl' and t < 0: return None
        if mode1 == 'Strecke' and not 0 <= s <= 1: return None
        if mode2 == 'Strecke' and not 0 <= t <= 1: return None
        return self.p.add(self.u.mult(s))
    
class Goal:
    def __init__(self, x):
        self.center = Vector(x,3250)
        d = Vector(0,1500)   
        p1 = self.center.sub(d)
        p2 = self.center.add(d)
        self.line = Gerade(p1, p2.sub(p1))
    
class Entity:
    def __init__(self, id, entity_type, x, y, vx, vy, state):
        self.pos = Vector(x,y)
        self.v = Vector(vx,vy)
        self.id = id
        self.type = entity_type
        self.state = state
        if entity_type == 'WIZARD' or entity_type == 'OPPONENT_WIZARD':
            self.radius = 400
            self.mass = 1
            self.friction = 0.75
        elif entity_type == 'SNAFFLE':
            self.radius = 150
            self.mass = 0.5
            self.friction = 0.75
        elif entity_type == 'BLUDGER':
            self.radius = 200
            self.mass = 8
            self.friction = 0.9
        else:
            raise Exception(f'unknown entity type {entity_type}')
        
    def __str__(self):
        return f'id={self.id} {self.pos} v={self.v} r={self.radius} {self.type[:3]}'
   
    def hasSnaffle(self):
        return self.state == 1
    
    def isGrabbed(self):
        return self.state == 1
    

In [4]:
def assignSnaffles(wizards, snaffles):
    ''' assigns snaffles to wizards 
    Für jeden der beiden wizards werden die beiden nächsten snaffles bestimmt
    Dann wird ausprobiert bei welchem Assignment die kürzeste Gesamtlänge herauskommt
    '''
    tmp = []
    for w in wizards:
        for sn in sorted(snaffles,key= lambda x: x.pos.dist(w.pos))[:2]:
            if sn not in tmp:
                tmp.append(sn)

    if len(tmp) == 1:
        return {wizards[i]:tmp[0] for i in range(2)}
    
    iter = it.permutations(tmp, r=2)
    best, best_val = None, None
    for tup in iter:     
        val = sum([wizards[i].pos.dist(tup[i].pos) for i in range(2)])
        if best_val is None or val < best_val:
            best = tup
            best_val = val

    return {wizards[i]:best[i] for i in range(2)}

In [ ]:
my_team_id = int(input())  # if 0 you need to score on the right of the map, if 1 you need to score on the left
if my_team_id == 0:
    mygoal, oppgoal = Goal(0),Goal(16000)
else:
    mygoal, oppgoal = Goal(16000),Goal(0)

while True:

    entities = []
    my_score, my_magic = [int(i) for i in input().split()]
    opponent_score, opponent_magic = [int(i) for i in input().split()]
    anz_entities = int(input())  # number of entities still in game

    for i in range(anz_entities):
        inputs = input().split()
        entity_id = int(inputs[0])  # entity identifier
        entity_type = inputs[1]  # "WIZARD", "OPPONENT_WIZARD" or "SNAFFLE" (or "BLUDGER" after first league)
        x = int(inputs[2])  # position
        y = int(inputs[3])  # position
        vx = int(inputs[4])  # velocity
        vy = int(inputs[5])  # velocity
        state = int(inputs[6])  # 1 if the wizard is holding a Snaffle, 0 otherwise
        entities.append(Entity(entity_id, entity_type, x, y, vx, vy, state))

    my_wizards = []
    opp_wizards = []
    snaffles = []
    bludgers = []

    for e in entities:
        if e.type == 'WIZARD':
            my_wizards.append(e)
        elif e.type == 'OPPONENT_WIZARD':
            opp_wizards.append(e)
        elif e.type == 'SNAFFLE':
            snaffles.append(e)
        elif e.type == 'BLUDGER':
            bludgers.append(e)

    actions = []
    assignment = assignSnaffles(my_wizards, snaffles)

    for k, w in enumerate(my_wizards):
        done = False

        ''' Throw '''
        if w.hasSnaffle():
            target = oppgoal.center.sub(w.v)
            action = f'THROW {target.x} {target.y} 500'
            actions.append(action)
            done = True

        '''  Flipendo '''
        if not done and  my_magic >= 20:
            for sn in snaffles:
                if sn.isGrabbed(): continue
                if sn.pos.dist(w.pos) > 9000: continue

                nextpos_sn = sn.pos.add(sn.v)
                nextpos_w = w.pos.add(w.v)
                g = Gerade(nextpos_w,nextpos_sn.sub(nextpos_w))
                sch = g.schnittpunkt(oppgoal.line,mode1='Strahl', mode2='Strecke')

                if sch is not None:
                    action = f'FLIPENDO {sn.id}'
                    actions.append(action)
                    done = True
                    break
                    
        ''' Accio '''
        if not done and my_magic >= 25:
            for sn in snaffles:
                if sn.isGrabbed(): continue
                if sn.pos.dist(w.pos) > 5000: continue
                if sn.pos.dist(oppgoal.center) < w.pos.dist(oppgoal.center): continue

                action = f'ACCIO {sn.id}'
                actions.append(action)
                done = True
                break

  
        ''' Move '''
        if not done:
            sn = assignment[w]
            if sn:
                target = sn.pos.add(sn.v)
                action = f'MOVE {target.x} {target.y} 150'
                actions.append(action)
         

    for action in actions:
        print(action)